In [1]:
#default_exp modules.featurenet

# Feature net
> As second step we implement the feature net. This will generate a pseudo-lidar image that can be fed into the
> backbone and thereafter into the detection head.


## 00 - Prerequesits

### 00.1 - Notebook imports

In [2]:
import sys
#sys.path.append("/home/qhs67/git/bachelorthesis_sven_thaele/code/")
sys.path.insert(0, "../pointpillars")
from utils.io import read_config
from data.dataset import VelDataset
from utils.time import time_method

### 00.2 - Imports

In [3]:
#export
import logging
import torch
import torch.nn as nn
import torch.nn.functional as F

logger = logging.getLogger(__name__)

## 01 - FeatureNet module
The actual module. As already said, the tensors from the pillar computation now will be converted into a pseudo image
of height H and width W.

In [4]:
#export
class FeatureNet(nn.Module):
    def __init__(self, fnet_cfg, pil_cfg):
        super(FeatureNet, self).__init__()

        # config data
        self.in_chan = fnet_cfg.getint("nb_features")
        self.out_chan = fnet_cfg.getint("nb_channels")
        self.max_ppp = pil_cfg.getint("max_points_per_pillar")
        self.fnet_cfg = fnet_cfg
        self.pil_cfg = pil_cfg

        # xy dimensions for pseudo image
        x_min = self.pil_cfg.getfloat("x_min")
        x_max = self.pil_cfg.getfloat("x_max")
        x_step = self.pil_cfg.getfloat("x_step")
        y_min = self.pil_cfg.getfloat("y_min")
        y_max = self.pil_cfg.getfloat("y_max")
        y_step = self.pil_cfg.getfloat("y_step")

        self.n_x = int((x_max - x_min) / x_step)
        self.n_y = int((y_max - y_min) / y_step)

        self.conv1 = nn.Conv2d(self.in_chan, self.out_chan, 1, bias=True)
        self.batchn = nn.BatchNorm2d(self.out_chan)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d([1, self.max_ppp])

    def forward(self, pil_batch: torch.tensor, ind_batch: torch.tensor):
        """Simplified point net to create a pseudo image from the point cloud."""
        if pil_batch.shape[0] != ind_batch.shape[0]:
            raise ValueError("The dimensions of the pillar batch and indices batch do not match.")
        # simplified PointNet
        # in: (N_b, D=9, N, P), out: (N_b, C_out, N, P)
        pil_batch = self.conv1(pil_batch)
        pil_batch = self.batchn(pil_batch)
        pil_batch = self.relu(pil_batch)
        # in: (N_b, C_out, N, P), out: (N_b, C_out, N)
        pil_batch = self.maxpool(pil_batch)

        n_b, n_c, n_x, n_y = pil_batch.shape

        # scatter points back to pillar position
        pse_img = torch.zeros([n_b,
                               n_c,
                               self.n_x,
                               self.n_y],
                               dtype=torch.float,
                               device="cuda:0")

        """#print(pse_img.shape, pil_batch.shape, ind_batch.shape)
        ind_batch = ind_batch.type(torch.LongTensor).unsqueeze(1).expand(-1, n_c, -1, -1)
        #print(pse_img.shape, pil_batch.shape, ind_batch.shape)
        #print(ind_batch[:,:,:,0].shape)
        ind_batch_tuple = torch.split(ind_batch, 1, dim=3)
        print(ind_batch_tuple[0].shape)
        pse_img[:,:,ind_batch_tuple[0], ind_batch_tuple[1]] = pil_batch
        print(pse_img.shape)

        return pse_img"""

        # flatten indices and pil_batch,
        # and also get the corresponding flat indices so torch.put() can be used
        batch_ind = torch.tensor(range(n_b), device=torch.device("cuda"))
        batch_ind = batch_ind.unsqueeze(1).expand(-1, n_c)
        batch_ind = batch_ind.unsqueeze(2).expand(-1, -1, n_x)

        feat_ind = torch.tensor(range(n_c), device=torch.device("cuda"))
        feat_ind = feat_ind.unsqueeze(0).expand(n_b, -1)
        feat_ind = feat_ind.unsqueeze(2).expand(-1, -1, n_x)

        ind_batch = ind_batch.unsqueeze(1).expand(-1, n_c, -1, -1)
        x_ind = ind_batch[:,:,:,0]
        y_ind = ind_batch[:,:,:,1]

        # calculate flat indices with batch, feature, x and y indices
        index = batch_ind * n_b + feat_ind * n_c + x_ind * n_x + y_ind
        index = index.long()

        index = torch.flatten(index)
        pil_batch = torch.flatten(pil_batch)

        return pse_img.put_(index, pil_batch)

        # old version
        #ind_x_batch = ind_batch.unsqueeze(1).expand(-1, pil_batch.shape[1], -1, -1)[:,:,:,0].unsqueeze(3).cuda()
        #ind_y_batch = ind_batch.unsqueeze(1).expand(-1, pil_batch.shape[1], -1, -1)[:,:,:,1].unsqueeze(3).cuda()
        #pse_img.scatter_(3, ind_x_batch, pil_batch)
        #pse_img.scatter_(2, ind_y_batch, pil_batch)
        #return pse_img[:,:,:self.n_x]


In [5]:
torch.cuda.empty_cache()
conf = read_config()
ds = VelDataset("/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/velodyne")
dl = torch.utils.data.DataLoader(ds, batch_size=2, num_workers=0)

def test():
    item = None
    for i, batch in enumerate(dl):
        if i == 1:
            break
        item = batch

    featurenet = FeatureNet(conf['featurenet'], conf['pillars'])
    featurenet.cuda()
    out = featurenet(item[0].cuda(), item[1].cuda())

time_method(test, runs=1)
#out, out.shape

torch.Size([2, 64, 12000, 1])
torch.Size([2, 12000, 2])
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]], device='cuda:0') torch.Size([2, 64, 12000])
tensor([[[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
         [ 2.,  2.,  2.,  ...,  2.,  2.,  2.],
         ...,
         [61., 61., 61.,  ..., 61., 61., 61.],
         [62., 62., 62.,  ..., 62., 62., 62.],
         [63., 63., 63.,  ..., 63., 63., 63.]],

        [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 1.,  1.,  1.,  ..., 

tensor(933.6871)